In [181]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [182]:
!pip install evaluate
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [183]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset

In [184]:
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [185]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [186]:
# 加载hf中dataset
ds = load_dataset('doushabao4766/msra_ner_k_V3')
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [187]:
# 原始文本转换模型需要token_idx,生成和token_idx对齐label
def data_input_proc(item):
    # 输入文本转换模型输入token索引
    #print(len(item['tokens']))
    input_data = {'input_ids': [], 'token_type_ids': [], 'attention_mask': [], 'labels': []}
    if len(item['tokens']) != 0 and len(item['tokens'])<=512:
        input_data = tokenizer(item['tokens'], truncation=True, add_special_tokens=False, max_length=1000)
        #'input_ids', 'token_type_ids', 'attention_mask'
        input_data['input_ids'] = torch.tensor(input_data['input_ids']).reshape(-1)
        input_data['token_type_ids'] = torch.tensor(input_data['token_type_ids']).reshape(-1)
        input_data['attention_mask'] = torch.tensor(input_data['attention_mask']).reshape(-1)
        input_data['labels'] = torch.tensor(item['ner_tags'])
        
    return input_data
    

ds2 = ds.map(data_input_proc, batched=False)  # batched 每次传入自定义方法样本数量多个

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [188]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [189]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

In [190]:
id2lbl = {'0': 'O','1': 'B-PER','2': 'I-PER','3': 'B-ORG','4': 'I-ORG','5': 'B-LOC','6': 'I-LOC'}
lbl2id = {id2lbl[tag]:tag for tag in id2lbl}

In [193]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[id2lbl[str(p)] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[id2lbl[str(l)] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

In [194]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [196]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds2['train'],
    eval_dataset=ds2['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [191]:
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=len(id2lbl),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [197]:
trainer.train()

Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.032100,0.028132,"{'precision': 0.9365994236311239, 'recall': 0.9229676961306355, 'f1': 0.9297335955658859, 'number': 2817}","{'precision': 0.836940836940837, 'recall': 0.8875286916602907, 'f1': 0.8614927590048272, 'number': 1307}","{'precision': 0.9416484318016046, 'recall': 0.9591381872213968, 'f1': 0.9503128450496872, 'number': 1346}",0.912886,0.923400,0.918113,0.991808
2,0.013500,0.027744,"{'precision': 0.9568810636004312, 'recall': 0.9453319133830316, 'f1': 0.9510714285714286, 'number': 2817}","{'precision': 0.8632416787264834, 'recall': 0.9127773527161438, 'f1': 0.8873187058386016, 'number': 1307}","{'precision': 0.9587628865979382, 'recall': 0.9673105497771174, 'f1': 0.9630177514792899, 'number': 1346}",0.933913,0.942962,0.938415,0.993157
3,0.005600,0.031963,"{'precision': 0.9534717251252685, 'recall': 0.9456869009584664, 'f1': 0.9495633576902512, 'number': 2817}","{'precision': 0.8826568265682657, 'recall': 0.9150726855394032, 'f1': 0.8985725018782871, 'number': 1307}","{'precision': 0.9529411764705882, 'recall': 0.962852897473997, 'f1': 0.9578713968957872, 'number': 1346}",0.935923,0.942596,0.939248,0.993336


Trainer is attempting to log a value of "{'precision': 0.9365994236311239, 'recall': 0.9229676961306355, 'f1': 0.9297335955658859, 'number': 2817}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.836940836940837, 'recall': 0.8875286916602907, 'f1': 0.8614927590048272, 'number': 1307}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9416484318016046, 'recall': 0.9591381872213968, 'f1': 0.9503128450496872, 'number': 1346}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9568810636004312, 'recall': 0.9453319133830316,

TrainOutput(global_step=4221, training_loss=0.022319054326805508, metrics={'train_runtime': 1931.0917, 'train_samples_per_second': 69.91, 'train_steps_per_second': 2.186, 'total_flos': 9129562022390832.0, 'train_loss': 0.022319054326805508, 'epoch': 3.0})

In [317]:
result = trainer.predict(ds2['test'])

In [329]:
print(ds['test'][0]['tokens'])
print(ds2['test'][0]['labels'])
print(result.label_ids[0])

['中', '共', '中', '央', '致', '中', '国', '致', '公', '党', '十', '一', '大', '的', '贺', '词', '各', '位', '代', '表', '、', '各', '位', '同', '志', '：', '在', '中', '国', '致', '公', '党', '第', '十', '一', '次', '全', '国', '代', '表', '大', '会', '隆', '重', '召', '开', '之', '际', '，', '中', '国', '共', '产', '党', '中', '央', '委', '员', '会', '谨', '向', '大', '会', '表', '示', '热', '烈', '的', '祝', '贺', '，', '向', '致', '公', '党', '的', '同', '志', '们', '致', '以', '亲', '切', '的', '问', '候', '！']
tensor([3, 4, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0,
        0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[   3    4    4    4    0    3    4    4    4    4    4    4    4    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    3
    4    4    4    4    4    4    4    4    4    4    4    4    4    4
    0    0    0    0    0    0    0    3    4    4    4  

In [434]:
#输入：“双方确定了今后发展中美关系的指导方针。”
#输出：[{"entity":"ORG","content":"中"},{"entity":"ORG","content":"美"}]
#{'0': 'O','1': 'B-PER','2': 'I-PER','3': 'B-ORG','4': 'I-ORG','5': 'B-LOC','6': 'I-LOC'}
def ner_func(sentense, id2lbl=id2lbl):
    l_s = list(sentense)
    input_data = tokenizer(l_s, truncation=True, add_special_tokens=False, max_length=512)
    input_data['input_ids'] = torch.tensor(input_data['input_ids']).reshape(-1)
    input_data['token_type_ids'] = torch.tensor(input_data['token_type_ids']).reshape(-1)
    input_data['attention_mask'] = torch.tensor(input_data['attention_mask']).reshape(-1)
    #input_data['labels'] = torch.zeros(len(sentense))
    #print([input_data])
    result = trainer.predict([input_data])
    #print(torch.tensor(result.predictions).reshape(-1,7))
    #print(torch.max(torch.tensor(result.predictions).reshape(-1,7),1))
    label_ids = torch.max(torch.tensor(result.predictions).reshape(-1,7),1).indices
    #print(result.predictions)
    #print(label_ids)
    out = []
    for i in range(len(label_ids)):
        key = label_ids[i].item()
        if(key != 0):
            label = id2lbl[str(key)][2:]
            out.append({"entity":id2lbl[str(key)][2:],"content":l_s[i]})
    return out

In [435]:
i = ner_func("双方确定了今后发展中美关系的指导方针。")
i

[{'entity': 'LOC', 'content': '中'}, {'entity': 'LOC', 'content': '美'}]